<a href="https://colab.research.google.com/github/KOJUNKYUNG/prbl_TIL/blob/master/ML/Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = load_breast_cancer()

In [ ]:
X = df.data
y = df.target
X.shape, y.shape

((569, 30), (569,))

In [ ]:
import math
model_rf = RandomForestClassifier(n_estimators = 10, random_state = 0)
def K_fold(X, y, k, model):
    n = len(X)//k
    idxs = []
    for i in range(k):
        try:
            idx = range(i*n, (i+1)*n)
            idxs.append(idx)
        except:
            break
    train_score = []
    test_score = []
    for i in range(k):
        test_y = y[idxs[i]]
        test_X = X[idxs[i]]
        train_X = np.delete(X, idxs[i], axis=0)
        train_y = np.delete(y, idxs[i], axis=0)
        
        model.fit(train_X, train_y)

        train_score.append(model.score(train_X, train_y))
        test_score.append(model.score(test_X, test_y))
    # print(train_score, test_score, idxs)
    print('train score :', np.mean(train_score))
    print('test score :', np.mean(test_score))

In [ ]:
import math
k = 5
model = RandomForestClassifier(n_estimators = 10, random_state = 0)
block_size = math.ceil(X.shape[0]/k)
score_list = []
for i in range(k):
    max_length = min(block_size*(i+1), X.shape[0])
    test_idxs = range(block_size*i, max_length)
    train_idxs = [idx for idx in range(X.shape[0]) if idx not in test_idxs]
    train_X = X[train_idxs]
    train_y = y[train_idxs]
    test_X = X[test_idxs]
    test_y = y[test_idxs]
    model.fit(train_X, train_y)
    score = model.score(test_X, test_y)
    score_list.append(score)
print(np.mean(score_list))

0.9420276354603322


In [ ]:
def kfold_train(estimator, X, y, k=5):
    block_size = math.ceil(X.shape[0]/k)
    score_list = []
    for i in range(k):
        max_length = min(block_size*(i+1), X.shape[0])
        test_idxs = range(block_size*i, max_length)
        train_idxs = [idx for idx in range(X.shape[0]) if idx not in test_idxs]
        train_X = X[train_idxs]
        train_y = y[train_idxs]
        test_X = X[test_idxs]
        test_y = y[test_idxs]
        estimator.fit(train_X, train_y)
        score = estimator.score(test_X, test_y)
        score_list.append(score)
    # print(score_list)
    avg_score = np.mean(score_list)
    return avg_score

In [ ]:
kfold_train(model, X, y, k=3)

[0.9263157894736842, 0.9578947368421052, 0.9629629629629629]


0.9490578297595841

## GridSearch

In [ ]:
grid_params = {'max_depth':[1, 2, 3, 4, 5, 6], 'min_samples_split':[2, 3]}
now_param = {'max_depth':1, 'min_samples_split':3}

In [ ]:
RandomForestClassifier(**now_param)

RandomForestClassifier(max_depth=1, min_samples_split=3)

In [ ]:
from itertools import product
# https://velog.io/@davkim1030/Python-%EC%88%9C%EC%97%B4-%EC%A1%B0%ED%95%A9-product-itertools

In [ ]:
now_param = {'max_depth':1, 'min_samples_split':3}

In [ ]:
list(product(*grid_params))

In [ ]:
grid_params.keys()

dict_keys(['max_depth', 'min_samples_split'])

In [ ]:
list(product(*(grid_params.values())))

[(1, 2),
 (1, 3),
 (2, 2),
 (2, 3),
 (3, 2),
 (3, 3),
 (4, 2),
 (4, 3),
 (5, 2),
 (5, 3),
 (6, 2),
 (6, 3)]

In [ ]:
from itertools import product
grid_params = {'max_depth':[1, 2, 3, 4, 5, 6], 'min_samples_split':[2, 3]}

In [ ]:
def rf_gridsc(X, y, grid_params, k):
    pram_pd = list(product(*(grid_params.values())))
    for ps in pram_pd:
        for i, key in enumerate(grid_params):
            now_param[key] = ps[i]
        
        model_rf = RandomForestClassifier(**now_param)
        print(ps, '결과 : ')
        K_fold(X, y, k, model_rf)
        print('\n')
        

In [ ]:
rf_gridsc(X, y, grid_params, 10)

In [ ]:
grid_parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2, 3]}
cv = 3

In [ ]:
from itertools import product
# list(product())
param = dict(zip(grid_parameters, (1, 4)))

In [ ]:
model = RandomForestClassifier(**param)
kfold_train(model, df.data, df.target, k=cv)

[0.8421052631578947, 0.9526315789473684, 0.9470899470899471]


0.9139422630650701

In [ ]:
for value in list(product(*grid_parameters.values())):
    param = dict(zip(grid_parameters, value))

    model = RandomForestClassifier(**param)
    print(kfold_train(model, df.data, df.target, k=cv))

[0.8421052631578947, 0.9473684210526315, 0.9365079365079365]
0.9086605402394875
[0.8368421052631579, 0.9473684210526315, 0.9365079365079365]
0.9069061542745752
[0.9, 0.9578947368421052, 0.9576719576719577]
0.9385222315046876
[0.9, 0.9736842105263158, 0.9523809523809523]
0.9420217209690893
[0.9157894736842105, 0.9736842105263158, 0.9470899470899471]
0.945521210433491
[0.9105263157894737, 0.9789473684210527, 0.9629629629629629]
0.9508122157244965


In [ ]:
def gridsearch_cv(model_class, grid_parameters, data, target, cv=3):
    grid_list = []
    for value in list(product(*grid_parameters.values())):
        param = dict(zip(grid_parameters, value))

        model = model_class(**param)
        score = kfold_train(model, data, target, k=cv)

        grid_list.append([param, score])
    return grid_list

In [ ]:
result = gridsearch_cv(RandomForestClassifier, grid_parameters, df.data, df.target, cv=5)
result

[0.7982456140350878, 0.9210526315789473, 0.9649122807017544, 0.9473684210526315, 0.9380530973451328]
[0.7894736842105263, 0.9210526315789473, 0.956140350877193, 0.9473684210526315, 0.9557522123893806]
[0.8771929824561403, 0.9385964912280702, 0.9736842105263158, 0.9473684210526315, 0.9646017699115044]
[0.8771929824561403, 0.9210526315789473, 0.9736842105263158, 0.956140350877193, 0.9646017699115044]
[0.8859649122807017, 0.956140350877193, 0.9736842105263158, 0.956140350877193, 0.9734513274336283]
[0.8859649122807017, 0.9298245614035088, 0.9736842105263158, 0.956140350877193, 0.9734513274336283]


[[{'max_depth': 1, 'min_samples_split': 2}, 0.9139264089427106],
 [{'max_depth': 1, 'min_samples_split': 3}, 0.9139574600217356],
 [{'max_depth': 2, 'min_samples_split': 2}, 0.9402887750349326],
 [{'max_depth': 2, 'min_samples_split': 3}, 0.9385343890700202],
 [{'max_depth': 3, 'min_samples_split': 2}, 0.9490762303990063],
 [{'max_depth': 3, 'min_samples_split': 3}, 0.9438130725042695]]